In [1]:
from dotenv import load_dotenv
import os

# Load document and start session 

In [2]:
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 

In [3]:
from h2ogpte import H2OGPTE

client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key= H2O_API_KEY,
)

# Create a new collection
collection_id = client.create_collection(
    name='First meeting transcirpt',
    description='Information related to the first group meeting',
)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.7`.
You can enable strict version checking by passing strict_version_check=True.


In [4]:
with open('meeting_mintues.txt', 'rb') as f:
    meeting_data1 = client.upload('meeting_mintues.txt', f)
client.ingest_uploads(collection_id, [meeting_data1,])

Job(id='b9cd4160-55dc-486b-8d3e-6051af96a65b', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 22, 14, 1, 32, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='16b14b9d57f94c39b4446ff557971f4c', status='Indexing done.'), JobStatus(id='7294d974664446aea89605922b1dca67', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 22, 14, 1, 59, tzinfo=TzInfo(UTC)), duration='27s')

In [6]:
chat_session_id = client.create_chat_session(collection_id)

In [7]:
llm_list = client.get_llms() # default use first elem, index[0]
for llm in llm_list:
    print(llm)

{'base_model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'prompt_type': 'mixtral', 'prompt_dict': {'promptA': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'promptB': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'PreInstruct': '<s> [INST] ', 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': '</s> ', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': 'You are an AI that follows instructions extremely well and as helpful as possible.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None,

In [8]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        reply = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm="h2oai/h2ogpt-4096-llama2-70b-chat",
        )
        return reply

# Summarizer and system prompt 

In [9]:
system_prompt = """
You are a Meeting Assistant for a software company. You will be given the meeting log in the form of audio, powerpoint 
or PDF. Your general duties include but are not limited to summarising important points during the meeting, taking note 
of everyone's deliverables and organising all information in a clear and concise form. Upon reading your output all
members should have a clear understanding of the meeting objectives, agenda, and outcomes.
"""

In [28]:
summarizer_prompt = """
Meeting Assistant Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.

2. Meeting Details:
    Date and Time: When did the meeting take place?
    Duration: How long did the meeting last?

3. Meeting Purpose: 
    Objective: What is the primary objective or purpose of the meeting?
    Desired Outcome: What outcomes do you hope to achieve by the end of the meeting?

4. Meeting summary: Provide a concise summary of the entire meeting by listing out all the main highlights
                    such that participants not in the meeting will be able to understand what happened during the meeting.
    
5. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible.

Present points 1-5 in the following format: 
{"Attendees": "", "Meeting Details": ["Date and Time":"", "Duration":""], 
"Meeting Details": ["Objective":"", "Desired Outcome":""], "Meeting summary":"", 
"Action Items": ["Deliverables":"", "Assign":""]
} 
"""

In [29]:
meeting_summary_response = rag_chat(client, chat_session_id, summarizer_prompt, system_prompt)
print(meeting_summary_response.content)

Attendees: Ryan_Edward, Eugene_YJ, Hannah Nga, Ben_CH, Jeremy_PH, Chan Yi Ru Micole

Meeting Details:
Date and Time: 22:18:02
Duration: Not specified

Meeting Purpose:
Objective: To discuss the development of a chatbot for a software company
Desired Outcome: To outline the key features and functionalities of the chatbot, and to assign tasks to team members for its development

Meeting Summary:
The meeting discussed the development of a chatbot for a software company. The team discussed the key features and functionalities of the chatbot, including the ability to summarize meetings, generate meeting minutes, and provide translations. The team also discussed the distribution of workload and assigned tasks to team members for the development of the chatbot.

Action Items:
Deliverables:

* Develop a chatbot for a software company
* Include the following features:
	+ Summarize meetings
	+ Generate meeting minutes
	+ Provide translations

Assign:

* Ryan_Edward: Lead the development of the c

# Sentiment Analysis

In [37]:
sentiment_prompt = """
Sentiment Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.
    
2. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible.
            For each work assigned arange them in order of importance
"""

In [38]:
meeting_summary_response = rag_chat(client, chat_session_id, sentiment_prompt, system_prompt)
print(meeting_summary_response.content)

Sure, I can help you with that. Here's the information you requested:

1. Attendees:

The following individuals attended the meeting:

* Ryan_Edward
* Chan Yi Ru Micole
* Hannah Nga
* Eugene_YJ
* Jeremy_PH
* Ben_CH

Their email addresses are not specified in the meeting logs provided.

2. Action Items:

a. Deliverables:

The meeting discussed the following deliverables:

* Meeting minute summary
* Actionable items
* Sentiment analysis
* Translation

b. Assignments:

The meeting assigned the following tasks to specific attendees:

* Ryan_Edward:
	+ Create a home page for users to upload files.
	+ Implement sentiment analysis.
	+ Incorporate translation.
	+ Regenerate the whole meeting.
* Chan Yi Ru Micole:
	+ Develop a pipeline.
	+ Implement sentiment analysis.
	+ Incorporate translation.
* Hannah Nga:
	+ Create a task list.
	+ Assign tasks to users.
	+ Determine priority allocation.
* Eugene_YJ:
	+ Implement automated follow-ups.
* Jeremy_PH:
	+ Work on the sentiment analysis.
* Ben_CH